例子：小明 100 米跑成绩：10.5 秒，1500 米跑成绩：3 分 20 秒，铅球成绩：12 米。他适合参加哪些搏击运动训练。

In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

client = OpenAI(
    api_key="sk-B9wonBTcGeUhTY3Rn3N0raK7jR8DQeKuJzxmhtOAdQXAnK2o",
    base_url="https://api.chatanywhere.tech"
)


In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature  # 模型输出的随机性，0 表示随机性最小
    )
    return response.choices[0].message.content

def performance_analyser(text):
    prompt = f"{text}\n请根据以上成绩，分析候选人在速度、耐力、力量三方面素质的分档。分档包括：强（3），中（2），弱（1）三档。\
                \n以JSON格式输出，其中key为素质名，value为以数值表示的分档。"
    response = get_completion(prompt)
    return json.loads(response)

def possible_sports(talent, category):
    prompt = f"需要{talent}强的{category}运动有哪些。给出10个例子，以array形式输出。确保输出能由json.loads解析。"
    response = get_completion(prompt, temperature=0.8)
    return json.loads(response)

def evaluate(sports, talent, value):
    prompt = f"分析{sports}运动对{talent}方面素质的要求: 强（3），中（2），弱（1）。\
                \n直接输出挡位数字。输出只包含数字。"
    response = get_completion(prompt)
    val = int(response)
    print(f"{sports}: {talent} {val} {value>=val}")
    return value >= val

def report_generator(name, performance, talents, sports):
    level = ['弱', '中', '强']
    _talents = {k: level[v-1] for k, v in talents.items()}
    prompt = f"已知{name}{performance}\n身体素质：{_talents}。\n生成一篇{name}适合{sports}训练的分析报告。"
    response = get_completion(prompt, model="gpt-3.5-turbo")
    return response

In [3]:
name = "小明"
performance = "100米跑成绩：10.5秒，1500米跑成绩：3分20秒，铅球成绩：12米。"
category = "搏击"
 
talents = performance_analyser(name+performance)
print("===talents===")
print(talents)

===talents===
{'速度': 3, '耐力': 2, '力量': 3}


In [4]:
cache = set()
# 深度优先

# 第一层节点
for k, v in talents.items():
    if v < 3:  # 剪枝
        continue
    leafs = possible_sports(k, category)
    print(f"==={k} leafs===")
    print(leafs)
    # 第二层节点
    for sports in leafs:
        if sports in cache:
            continue
        cache.add(sports)
        suitable = True
        for t, p in talents.items():
            if t == k:
                continue
            # 第三层节点
            if not evaluate(sports, t, p):  # 剪枝
                suitable = False
                break
        if suitable:
            report = report_generator(name, performance, talents, sports)
            print("****")
            print(report)
            print("****")

===速度 leafs===
['拳击', '泰拳', '跆拳道', '空手道', '综合格斗', '散打', '击剑', '空手搏击', '巴西柔术', '自由搏击']
拳击: 耐力 3 False
泰拳: 耐力 3 False
跆拳道: 耐力 3 False
空手道: 耐力 3 False
综合格斗: 耐力 3 False
散打: 耐力 3 False
击剑: 耐力 3 False
空手搏击: 耐力 3 False
巴西柔术: 耐力 3 False
自由搏击: 耐力 3 False
===力量 leafs===
['拳击', '综合格斗', '泰拳', '踢拳', '巴西柔术', '柔道', '空手道', '泰拳', '跆拳道', '自由搏击']
踢拳: 速度 3 True
踢拳: 耐力 3 False
柔道: 速度 3 True
柔道: 耐力 3 False


原代码分析

In [12]:
pwd # type: ignore

'f:\\Project_LLM\\LLM_FirstLearn\\ToT_Persist'

In [19]:
import os
import json
import argparse
from email.policy import default
from sympy import false # type: ignore

from task.get_task import  get_task
# from methods.bfs import solve, naive_solve
# solve : 主求解函数，执行生成、评估和选择步骤，最终返回最佳输出。
# naive_solve : 一个简单的求解函数，直接生成样本并返回
from config.model import gpt_usage

# from methods.bfs import  solve, naive_solve



